In [ ]:
import torch
import numpy as np
import cv2
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms

# Load model FaceNet pretrained
model = InceptionResnetV1(pretrained='vggface2').eval()

# Tiền xử lý ảnh
def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((160, 160)),  
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
    return transform(image).unsqueeze(0)

# Tính embedding của ảnh
def get_embedding(image_path):
    image_tensor = preprocess_image(image_path)
    with torch.no_grad():
        embedding = model(image_tensor)
    return embedding

# Tính khoảng cách cosine
def cosine_similarity(emb1, emb2):
    emb1 = emb1 / emb1.norm(p=2)
    emb2 = emb2 / emb2.norm(p=2)
    return torch.dot(emb1.squeeze(0), emb2.squeeze(0)).item()

# So sánh 2 ảnh
def compare_faces(img1_path, img2_path, threshold=0.6):
    emb1 = get_embedding(img1_path)
    emb2 = get_embedding(img2_path)
    similarity = cosine_similarity(emb1, emb2)
    print(f"Cosine Similarity: {similarity:.4f}")
    
    if similarity > threshold:
        print("→ Hai ảnh là cùng một người.")
    else:
        print("→ Hai ảnh là hai người khác nhau.")

# Chạy thử
img1 = "path_to_image1.jpg"
img2 = "path_to_image2.jpg"
compare_faces(img1, img2)
